# Config

In [1]:
source("/home/vickydiliscia/dmeyf2025/src/competencia/experimento/config.r")
source("/home/vickydiliscia/dmeyf2025/src/competencia/experimento/utils/04_entrenar_modelos.R")
source("/home/vickydiliscia/dmeyf2025/src/competencia/experimento/utils/05_ensamblar_promedio.R")
source("/home/vickydiliscia/dmeyf2025/src/competencia/experimento/utils/09_armar_archivo_envios.R")
source("/home/vickydiliscia/dmeyf2025/src/competencia/experimento/utils/10_enviar_kaggle.R")

## Semillas
PARAM <- list()
PARAM$semilla_primigenia <- SEEDS[1]
PARAM$experimento <- 5001
# training y future
PARAM$train <- c(202102)
PARAM$train_final <- c(202101,202102,202103,202104)
PARAM$future <- c(202104)
PARAM$future_final <- c(202106)
PARAM$semilla_kaggle <- 314159
PARAM$cortes <- seq(6000, 19000, by= 500)

# undersampling
PARAM$trainingstrategy$undersampling <- 0.3

# carpeta de trabajo
setwd("/content/buckets/b1/exp")
experimento_folder <- paste0("HT", PARAM$experimento)
dir.create(experimento_folder, showWarnings=FALSE)
setwd( paste0("/content/buckets/b1/exp/", experimento_folder ))

## Librerias

In [2]:
# cargo las librerias que necesito
require("data.table")
require("parallel")

if(!require("R.utils")) install.packages("R.utils")
require("R.utils")

if( !require("primes") ) install.packages("primes")
require("primes")

if( !require("utils") ) install.packages("utils")
require("utils")

if( !require("rlist") ) install.packages("rlist")
require("rlist")

if( !require("yaml")) install.packages("yaml")
require("yaml")

if( !require("lightgbm") ) install.packages("lightgbm")
require("lightgbm")

if( !require("DiceKriging") ) install.packages("DiceKriging")
require("DiceKriging")

if( !require("mlrMBO") ) install.packages("mlrMBO")
require("mlrMBO")

Loading required package: parallel

Loading required package: R.utils

Loading required package: R.oo

Loading required package: R.methodsS3

R.methodsS3 v1.8.2 (2022-06-13 22:00:14 UTC) successfully loaded. See ?R.methodsS3 for help.

R.oo v1.27.1 (2025-05-02 21:00:05 UTC) successfully loaded. See ?R.oo for help.


Attaching package: ‘R.oo’


The following object is masked from ‘package:R.methodsS3’:

    throw


The following objects are masked from ‘package:methods’:

    getClasses, getMethods


The following objects are masked from ‘package:base’:

    attach, detach, load, save


R.utils v2.13.0 (2025-02-24 21:20:02 UTC) successfully loaded. See ?R.utils for help.


Attaching package: ‘R.utils’


The following object is masked from ‘package:utils’:

    timestamp


The following objects are masked from ‘package:base’:

    cat, commandArgs, getOption, isOpen, nullfile, parse, use, warnings


Loading required package: primes

Loading required package: rlist

Loading required packa

## Parametros LGBM

In [3]:
PARAM$hyperparametertuning$xval_folds <- 5

# parametros fijos del LightGBM que se pisaran con la parte variable de la BO
PARAM$lgbm$param_fijos <-  list(
  boosting= "gbdt", # puede ir  dart  , ni pruebe random_forest
  objective= "binary",
  metric= "auc",
  first_metric_only= FALSE,
  boost_from_average= TRUE,
  feature_pre_filter= FALSE,
  force_row_wise= TRUE, # para reducir warnings
  verbosity= -100,

  seed= PARAM$semilla_primigenia,

  max_depth= -1L, # -1 significa no limitar,  por ahora lo dejo fijo
  min_gain_to_split= 0, # min_gain_to_split >= 0
  min_sum_hessian_in_leaf= 0.001, #  min_sum_hessian_in_leaf >= 0.0
  lambda_l1= 0.0, # lambda_l1 >= 0.0
  lambda_l2= 0.0, # lambda_l2 >= 0.0
  max_bin= 31L, # lo debo dejar fijo, no participa de la BO

  bagging_fraction= 1.0, # 0.0 < bagging_fraction <= 1.0
  pos_bagging_fraction= 1.0, # 0.0 < pos_bagging_fraction <= 1.0
  neg_bagging_fraction= 1.0, # 0.0 < neg_bagging_fraction <= 1.0
  is_unbalance= FALSE, #
  scale_pos_weight= 1.0, # scale_pos_weight > 0.0

  drop_rate= 0.1, # 0.0 < neg_bagging_fraction <= 1.0
  max_drop= 50, # <=0 means no limit
  skip_drop= 0.5, # 0.0 <= skip_drop <= 1.0

  extra_trees= FALSE,

  num_iterations= 1200,
  learning_rate= 0.02,
  feature_fraction= 0.5,
  num_leaves= 750,
  min_data_in_leaf= 5000
)

In [4]:
kbayesiana <- "bayesiana.RDATA"  # tu archivo con el opt.state adentro
# Cargar el archivo (esto mete en memoria los objetos guardados, p. ej. opt.state)
load(kbayesiana)

# Si el archivo contiene el estado de la optimización:
bayesiana_salida <- list(opt.path = opt.state$opt.path)
                         
tb_bayesiana <- as.data.table(bayesiana_salida$opt.path)
colnames( tb_bayesiana)

# almaceno los resultados de la Bayesian Optimization
# y capturo los mejores hiperparametros encontrados

tb_bayesiana <- as.data.table(bayesiana_salida$opt.path)

tb_bayesiana[, iter := .I]

# ordeno en forma descendente por AUC = y
setorder(tb_bayesiana, -y)

# grabo para eventualmente poder utilizarlos en OTRA corrida
fwrite( tb_bayesiana,
  file= "BO_log.txt",
  sep= "\t"
)

# los mejores hiperparámetros son los que quedaron en el registro 1 de la tabla
PARAM$out$lgbm$mejores_hiperparametros <- tb_bayesiana[
  1, # el primero es el de mejor AUC
  setdiff(colnames(tb_bayesiana),
    c("y","dob","eol","error.message","exec.time","ei","error.model",
      "train.time","prop.type","propose.time","se","mean","iter")),
  with= FALSE
]

PARAM$out$lgbm$y <- tb_bayesiana[1, y]

write_yaml( PARAM, file="PARAM.yml")


print(PARAM$out$lgbm$mejores_hiperparametros)
print(PARAM$out$lgbm$y)

[1] "num_iterations"   "learning_rate"    "feature_fraction" "num_leaves"      
 [5] "min_data_in_leaf" "y"                "dob"              "eol"             
 [9] "error.message"    "exec.time"        "ei"               "error.model"     
[13] "train.time"       "prop.type"        "propose.time"     "se"              
[17] "mean"

   num_iterations learning_rate feature_fraction num_leaves min_data_in_leaf
            <int>         <num>            <num>      <int>            <int>
1:           1375     0.0101999        0.2633874        575              325
[1] 0.92906


# Tomar prediccion

In [18]:
setwd("/content/buckets/b1/exp")
experimento <- paste0("exp", PARAM$experimento)
dir.create(experimento, showWarnings= FALSE)
setwd( paste0("/content/buckets/b1/exp/", experimento ))
tag <- "5001_us03_lag1"

# Construí el path relativo al working dir actual
pred_file <- file.path("predicciones", paste0("prediccion_", tag, ".txt"))

# Verificá que exista
stopifnot(file.exists(pred_file))

tb_prediccion <- fread(pred_file, sep = "\t")  # fread detecta el separador solo


In [23]:
res <- lapply(PARAM$cortes, function(corte) {
  armar_archivo_envios(
    pred         = pred_file,
    corte        = corte,
    experimento  = "5001_us03_lag1",  # o PARAM$experimento
    out_dir      = "kaggle",
    id_col       = "numero_de_cliente",
    prob_col     = "prob",
    sep_in       = "\t"
  )
})

✅ Generado: kaggle/KA5001_us03_lag1_6000.csv (corte=6000)

✅ Generado: kaggle/KA5001_us03_lag1_6500.csv (corte=6500)

✅ Generado: kaggle/KA5001_us03_lag1_7000.csv (corte=7000)

✅ Generado: kaggle/KA5001_us03_lag1_7500.csv (corte=7500)

✅ Generado: kaggle/KA5001_us03_lag1_8000.csv (corte=8000)

✅ Generado: kaggle/KA5001_us03_lag1_8500.csv (corte=8500)

✅ Generado: kaggle/KA5001_us03_lag1_9000.csv (corte=9000)

✅ Generado: kaggle/KA5001_us03_lag1_9500.csv (corte=9500)

✅ Generado: kaggle/KA5001_us03_lag1_10000.csv (corte=10000)

✅ Generado: kaggle/KA5001_us03_lag1_10500.csv (corte=10500)

✅ Generado: kaggle/KA5001_us03_lag1_11000.csv (corte=11000)

✅ Generado: kaggle/KA5001_us03_lag1_11500.csv (corte=11500)

✅ Generado: kaggle/KA5001_us03_lag1_12000.csv (corte=12000)

✅ Generado: kaggle/KA5001_us03_lag1_12500.csv (corte=12500)

✅ Generado: kaggle/KA5001_us03_lag1_13000.csv (corte=13000)

✅ Generado: kaggle/KA5001_us03_lag1_13500.csv (corte=13500)

✅ Generado: kaggle/KA5001_us03_lag1_1400

In [28]:
source("/home/vickydiliscia/dmeyf2025/src/competencia/experimento/utils/10_enviar_kaggle.R")

In [29]:
COMP_NAME <- "dm-ey-f-2025-primera"  # p.ej.: "meic-dmeyf-2025"
pred_file <- file.path("predicciones", paste0("prediccion_", tag, ".txt"))

# Un envío de prueba (top 7500)
enviar_archivo(
  pred          = pred_file,
  corte         = 7500,
  experimento   = "5001_us03_lag1",
  competition   = COMP_NAME,
  keep_file     = TRUE,          # deja el CSV en /kaggle
  out_dir       = "kaggle"
)


⬆️ Subiendo: kaggle/KA5001_us03_lag1_7500.csv 
 kaggle competitions submit -c dm-ey-f-2025-primera -f "kaggle/KA5001_us03_lag1_7500.csv" -m "Exp=5001_us03_lag1 | Corte=7500 | submit auto" 
Successfully submitted to DMEyF 2025 Primera 

📜 Últimas submissions:
fileName                    date                        description                                    status                     publicScore  privateScore  
--------------------------  --------------------------  ---------------------------------------------  -------------------------  -----------  ------------  
KA5001_us03_lag1_7500.csv   2025-10-09 21:54:34.647000  Exp=5001_us03_lag1 | Corte=7500 | submit auto  SubmissionStatus.PENDING                              
KA5001_us03_lag1_7000.csv   2025-10-09 21:45:14.693000  Exp=5001_us03_lag1 | Corte=7000 | submit auto  SubmissionStatus.ERROR                                
KA5001_us03_lag1_11000.csv  2025-10-09 21:17:42.420000  KA5001_us03_lag1_11000                         Submi

In [30]:
comp_name <- "dm-ey-f-2025-primera"  # p.ej.: "meic-dmeyf-2025"
cortes <- PARAM$cortes
tag <- "5001_us03_lag1"
pred_file <- file.path("predicciones", paste0("prediccion_", tag, ".txt"))

# Loop para enviar cada uno
for (c in cortes) {
  enviar_archivo(
    pred         = pred_file,  # o tu data.table con las probs
    corte        = c,
    experimento  = tag,
    competition  = comp_name,    # slug exacto de Kaggle
    message_submit = sprintf("Exp=exp_lags1 | Corte=%d", c)
  )
}


⬆️ Subiendo: kaggle/KA5001_us03_lag1_6000.csv 
 kaggle competitions submit -c dm-ey-f-2025-primera -f "kaggle/KA5001_us03_lag1_6000.csv" -m "Exp=exp_lags1 | Corte=6000" 
Successfully submitted to DMEyF 2025 Primera 

📜 Últimas submissions:
fileName                    date                        description                                    status                     publicScore  privateScore  
--------------------------  --------------------------  ---------------------------------------------  -------------------------  -----------  ------------  
KA5001_us03_lag1_6000.csv   2025-10-09 21:58:02.927000  Exp=exp_lags1 | Corte=6000                     SubmissionStatus.PENDING                              
KA5001_us03_lag1_7500.csv   2025-10-09 21:54:34.647000  Exp=5001_us03_lag1 | Corte=7500 | submit auto  SubmissionStatus.COMPLETE  2.000                      
KA5001_us03_lag1_7000.csv   2025-10-09 21:45:14.693000  Exp=5001_us03_lag1 | Corte=7000 | submit auto  SubmissionStatus.ERROR  